In [1]:
import time
import os

try:
    from watchdog.observers import Observer
    from watchdog.events import FileSystemEventHandler
except ModuleNotFoundError as e:
    print (e)
    os.system("pip install watchdog")

# ------------------------------------------------
class Handler(FileSystemEventHandler):
    def on_created(self, event): # 파일 생성시
        print (f'event type : {event.event_type}\n'
               f'event src_path : {event.src_path}')
        if event.is_directory:
            Directory = os.path.split(os.path.basename(event.src_path))[0]
            print (Directory," 디렉토리 생성")
        else: # not event.is_directory
            Extension = os.path.splitext(os.path.basename(event.src_path))[1]
            if Extension == '.csv':
                print ("csv 파일이 추가되었습니다.")
        
        print("여기다 파이썬 파일 실행하는 터미널 명령어 작성")

    def on_deleted(self, event):
        print ("삭제 이벤트 발생")

    def on_moved(self, event): # 파일 이동시
        print (f'event type : {event.event_type}\n')

class Watcher:
    # 생성자
    def __init__(self, baseDir):
        print ("감시 중 ...")
        self.event_handler = None      # Handler
        self.observer = Observer()     # Observer 객체 생성
        self.usersDirList = self.findCsvDir(baseDir) 
        self.target_directory = self.usersDirList   # 감시대상 경로
#         self.currentDirectorySetting() # instance method 호출 func(1)

#     # func (1) 현재 작업 디렉토리
#     def currentDirectorySetting(self):
#         print ("====================================")
#         print ("현재 작업 디렉토리:  ", end=" ")
#         os.chdir(self.target_directory)
#         print ("{cwd}".format(cwd = os.getcwd()))
#         print ("====================================")

    # func (2)
    def run(self):
        self.event_handler = Handler() # 이벤트 핸들러 객체 생성
        self.observer.schedule(
            self.event_handler,
            self.target_directory,
            recursive=False
        )

        self.observer.start() # 감시 시작
        try:
            while True: # 무한 루프
                time.sleep(1) # 1초 마다 대상 디렉토리 감시
        except KeyboardInterrupt as e: # 사용자에 의해 "ctrl + z" 발생시
            print ("감시 중지...")
            self.observer.stop() # 감시 중단
            
    # 캐시 파일 등은 제외하고 디렉토리만 리스트로 추출하는 함수
    def findDir(self, pwd):
        directory = []
        try:
            totalDir = os.listdir(pwd)
        except Exception as e:
            print(e)

        for eachDir in totalDir:
            fullDir = os.path.join(pwd, eachDir)
            if os.path.isdir(fullDir):
                directory.append(eachDir)

        return directory

    # 입력 받은 디렉토리(pwd)와 그 하위 디렉토리 목록(subDir)을 입력 받아 경로를 merge 시켜 리스트로 반환
    def mergeSubDirList(self, pwd, subDir):
        reDefinedDir = []

        for eachDir in subDir:
            reDefinedDir.append(os.path.join(pwd, eachDir))

        return reDefinedDir

    # 대상 경로를 입력해주면 해당 경로의 하위의 하위 디렉토리까지 리스트로 반환
    def findTreeDir(self, pwd):
        # 인풋 파라미터의 디렉토리 내의 디렉토리를 리스트로 만들어 병합
        subDir = self.findDir(pwd)
        subDirList = self.mergeSubDirList(pwd, subDir)

        # 위 디렉토리 리스트 내의 디렉토리를 다시 한 번 리스트로 만들어 병합
        treeDirList = []
        for each in subDirList:
            subDirList = findDir(each)
            treeDirList.append(self.mergeSubDirList(each, subDirList))

        return treeDirList

    # 최신 디렉토리만 리스트로 반환
    def findLatest(self, categories, index):
        result = []

        for eachList in categories:
            eachList.sort()
            try:
                result.append(eachList[-index])
            except Exception as e:
                result.append(eachList[0])
                print(e)

        return result

    # 입력 받은 디렉토리에서 CSV 파일만 전체 디렉토리 리스트로 추출
    def findCsv(self, pwd):
        try:
            fileList = os.listdir(pwd)
            csvList = [file for file in fileList if (file.endswith(".csv")) \
                       and (file.split("\\")[-1].split(".")[0] == "com")\
                        and (re.search("(food|breath|reward|hist|recommend)", file) is None)]
        except Exception as e:
            print(e)

        return csvList

    # 입력 받은 디렉토리 내에 csv 파일을 찾아 전체 디렉토리를 리스트로 반환
    def findCsvDir(self, pwd):
        # 디렉토리 내 csv 파일을 리스트로 반환
        try:
            csvList = self.findCsv(pwd)
        except Exception as e:
            print(e)
        try:
            csvPathList = self.mergeSubDirList(pwd, csvList)
        except Exception as e:
            print(e)

        return csvPathList

myWatcher = Watcher("./HealthCare/")
myWatcher.run()

감시 중 ...
현재 작업 디렉토리:   /Users/saebyul/OneDrive - 한국폴리텍대학/gitStudy/Python_Study/lecture/TeamHealthCareProject/HealthCare


FileNotFoundError: [Errno 2] No such file or directory: './HealthCare'